In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import patches

from field_util import Field, CartCoord
from astropy.cosmology import FlatLambdaCDM
from astropy.constants import c
import astropy.units as u

In [ ]:
with open("../Catalogs/all_specz_DM_v1.dat") as f:
    lines = f.readlines()
    headers = lines[0].split()
    data = []
    for line in lines[1:]:
        entry = line.split()
        data.append(entry)
gal_df = pd.DataFrame(data, columns = headers)
gal_df['RA'] = gal_df['RA'].astype(float)
gal_df['Dec'] = gal_df['Dec'].astype(float)
gal_df['zspec'] = gal_df['zspec'].astype(float)

In [ ]:
cosmo = FlatLambdaCDM(H0=100* u.km / u.s / u.Mpc, Om0=0.315) # Metin original
mycoord = CartCoord(150.14205192829834, 2.224237689411875, cosmo) # Used in Cosmic Birth

In [ ]:
x, y, z = mycoord.orig_to_box(gal_df['RA'], gal_df['Dec'], gal_df['zspec'])
x_pc, y_pc, z_pc = mycoord.orig_to_box(150.11, 2.161, 2.298)

gal_df['trans_dist'] = (np.sqrt((y - y_pc)**2 + (z - z_pc)**2)).value
gal_df['LOSV_diff'] = (gal_df['zspec'] - 2.298) * 3 * 1E5
gal_df['x'] = x.value
gal_df['y'] = y.value
gal_df['z'] = z.value

In [ ]:
# transverse plane: R < 4 Mpc/h
a = gal_df['trans_dist'] < 6
# LOS: vel diff < 600 km/s
b = np.abs(gal_df['LOSV_diff']) < 500
gal_member = gal_df[a&b]

In [ ]:
# iteratively find the center of pc
# inital guess
RA = 150.11
DE = 2.161
red = 2.298

# loop
x_pc, y_pc, z_pc = mycoord.orig_to_box(RA, DE, red)

for _ in range(100):

    gal_df['trans_dist'] = (np.sqrt((gal_df['y'] - y_pc)**2 + (gal_df['z']  - z_pc)**2))
    gal_df['LOSV_diff'] = (gal_df['zspec'] - red) * 3 * 1E5
    
    a = gal_df['trans_dist'] < 6
    # LOS: vel diff < 500 km/s
    b = np.abs(gal_df['LOSV_diff']) < 600

    gal_member = gal_df[a&b]
    x_pc = np.median(gal_member['x'])
    y_pc = np.median(gal_member['y'])
    z_pc = np.median(gal_member['z'])
    skycoord = mycoord.original_pos(x_pc, y_pc, z_pc)
    red = skycoord.distance.value
    print(x_pc, y_pc, z_pc, red, len(gal_member))

In [ ]:
from astropy.constants import G, M_sun
from astropy import units as u
import numpy as np

# estimate virial mass with NMAD and original method in the galaxy cluster paper.
def NMAD(data):
    # calculate NMAD
    # NMAD = 1.4826 * MAD
    # MAD = median(data - median(data))
    MAD = np.median(np.abs(data - np.median(data)))
    return 1.4826 * MAD

def Virial_Mass_NMAD(y_data, z_data, y_center, z_center, LOSV):
    V_est = NMAD(LOSV)
    R_list = np.sqrt((y_data - y_center)**2 + (z_data - z_center)**2)
    R_est = NMAD(R_list[R_list!=0])

    Mass_est = 3*np.pi/2*R_est*u.Mpc * (V_est*u.km/u.s)**2 / G / M_sun
    return Mass_est+0

def Virial_Mass_Girardi(gal_y, gal_z, gal_red):
    s = len(gal_y)
    gal_vel = gal_red * 3E5
    y_mat = np.array(np.repeat(gal_y, s)).reshape(s, s)
    z_mat = np.array(np.repeat(gal_z, s)).reshape(s, s)
    dist_mat = ((y_mat - y_mat.T)**2 + (z_mat - z_mat.T)**2)**(.5)
    R_PV = s*(s-1)/np.triu(dist_mat**-1, k=1).sum()
    sigma_P = np.sum((gal_vel - np.mean(gal_vel))**2 / (s-1))**.5
    M_viral = 3*np.pi/2 * (sigma_P*u.km/u.s)**2 * R_PV*u.Mpc  / G / M_sun+0
    return R_PV, sigma_P, M_viral

In [ ]:
# call this function to get result 
Virial_Mass_Girardi(gal_member['y_pro'], gal_member['z_pro'], gal_member['zspec'])

# in our case it is
# (0.40363864265716043, 361.4098555087836, <Quantity 5.77660621e+13>)

In [ ]:
# further calculate M200
from colossus.halo import mass_so
mass_so.M_to_R(5.78e13, 2.3, '200m')
# 281.20024764902956